In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Simple Chain

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {politician}")

chain = prompt | chatModel | StrOutputParser()

# Stream and print clean text
for chunk in chain.stream({"politician": "Donald Trump"}):
    print(chunk, end="", flush=True)

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')


One curious fact about Donald Trump is that he was the first U.S

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')


. president to have never held a political office or served in the military prior to his election in 2016. His background was primarily in business and entertainment, which set him apart from his predecessors and contributed to his unconventional approach to politics.

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"error":"Forbidden"}\\n\')')
Failed t

## LCEL
* LCEL has become the backbone of the newest versions of LangChain.
* Traditional chains are still supported, but treated as "Legacy" and have less functionality than the new LCEL chains.
* Many students struggle with LCEL.

### Main goals of LCEL
* Make it easy to build chains in a compact way.
* Support advanced LangChain functionality.

### Legacy Chain vs. LCEL Chain

#### Legacy

In [6]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

traditional_chain = LLMChain(
    llm=chatModel,
    prompt=prompt
)

traditional_chain.predict(soccer_player="Maradona")

c:\Users\xenow\Downloads\MyGitRepos\ownLangChainNotes\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


'One curious fact about Diego Maradona is that he was famously known for his "Hand of God" goal during the 1986 FIFA World Cup quarter-final match against England. However, what many people might not know is that Maradona also scored one of the greatest goals in World Cup history in the same match. Just minutes after the controversial Hand of God goal, he dribbled past five England players over a distance of 60 meters to score what is often referred to as the "Goal of the Century." This incredible display of skill and determination showcased his extraordinary talent and remains a defining moment in football history.'

### LCEL Chain
* The "pipe" operator `|` is the main element of the LCEL chains.
* The order (left to right) of the elements in a LCEL chain matters.
* An LCEL Chain is a Sequence of Runnables.

* All the components of the chain are Runnables.
* When we write chain.invoke() we are using invoke with all the componentes of the chain in an orderly manner:
    * First, we apply .invoke() to the prompt.
    * Then, with the previous output, we apply .invoke() to the model.
    * And finally, with the previous output, we apply .invoke() to the output parser.

In [8]:
chain = prompt | chatModel | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'A curious fact about Cristiano Ronaldo is that he has a unique goal celebration known as the "Siii!" celebration, which involves him jumping into the air, twisting in mid-air, and landing while shouting "Siii!" (which means "Yes!" in Spanish). This celebration has become iconic among fans and is often mimicked by young soccer players around the world. Interestingly, Ronaldo first popularized this celebration while playing for Real Madrid, and it has since become synonymous with his brand and personality on and off the pitch.'

In [9]:
# Stream and print clean text
for chunk in chain.stream({"soccer_player": "Ronaldo"}):
    print(chunk, end="", flush=True)

One curious fact about Cristiano Ronaldo is that he has a unique dietary regimen that includes a strict focus on protein and hydration. He reportedly eats six small meals a day, which often include foods like chicken, fish, vegetables, and whole grains. Additionally, he avoids sugary drinks and prefers to stay hydrated with water. Ronaldo's commitment to his diet is part of what he believes contributes to his longevity and performance in professional football, allowing him to maintain an elite level of fitness even into his late 30s.

### LCEL: The runnable execution order

![alt text](./lcel-2.png)

* All the components of the chain are Runnables.
* **When we write chain.invoke() we are using invoke with all the componentes of the chain in an orderly manner**:
    * First, we apply .invoke() with the user input to the prompt template.
    * Then, with the completed prompt, we apply .invoke() to the model.
    * And finally, we apply .invoke() to the output parser with the output of the model.
* IMPORTANT: the order of operations in a chain matters. If you try to execute the previous chain with the components in different order, the chain will fail.

#### Invoke vs Stream vs Batching

LCEL Chains/Runnables are used with:
* chain.invoke(): call the chain on an input.
* chain.stream(): call the chain on an input and stream back chunks of the response.
* chain.batch(): call the chain on a list of inputs.

In [10]:
prompt = ChatPromptTemplate.from_template("Tell me one sentence about {politician}.")
chain = prompt | chatModel

In [11]:
# invoke
chain.invoke({"politician": "Churchill"})

AIMessage(content='Winston Churchill was a British statesman, military leader, and prime minister known for his leadership during World War II and his powerful speeches that inspired resilience in the face of adversity.', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 14, 'total_tokens': 50, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run-d22af371-7620-49f2-b5c7-7a9a08d7d5e8-0', usage_metadata={'input_tokens': 14, 'output_tokens': 36, 'total_tokens': 50})

In [12]:
# stream
for s in chain.stream({"politician": "F.D. Roosevelt"}):
    print(s.content, end="", flush=True)

Franklin D. Roosevelt was the 32nd President of the United States, serving from 1933 to 1945, and is best known for leading the country through the Great Depression and World War II while implementing the New Deal to promote economic recovery and social reform.

In [13]:
# batch
chain.batch([{"politician": "Lenin"}, {"politician": "Stalin"}])

[AIMessage(content='Vladimir Lenin was a revolutionary leader who played a key role in the Russian Revolution of 1917 and the establishment of the Soviet Union, advocating for Marxist principles and the idea of a vanguard party to guide the working class.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 14, 'total_tokens': 61, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0857daf-6eae-443d-8ad7-84d55672865b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 47, 'total_tokens': 61}),
 AIMessage(content='Joseph Stalin was the leader of the Soviet Union from the mid-1920s until his death in 1953, known for his totalitarian regime, rapid industrialization, and widespr

#### LCEL Chains/Runnables can be also used asynchronously:
* chain.ainvoke(): call the chain on an input.
* chain.astream(): call the chain on an input and stream back chunks of the response.
* chain.abatch(): call the chain on a list of inputs.

### RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [14]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

chain.invoke("Abram")

'Abram'

### RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [15]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [16]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("Abram")

'Abramovich'

### RunnableParallel (Important)
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [19]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [20]:
chain.invoke("Abram")

{'operation_a': 'Abram', 'operation_b': 'Abramovich'}

* Instead of using RunnableLambda, now we are going to use a lambda function and we will invoke the chain with two inputs:

In [21]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": lambda x: x["name"]+"ovich"
    }
)

In [25]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

{'operation_a': {'name1': 'Jordam', 'name': 'Abram'},
 'soccer_player': 'Abramovich'}

* See how the lambda function is taking the "name" input.

#### We can add more Runnables to the chain
* In the following example, the prompt Runnable will take the output of the RunnableParallel:

In [26]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [27]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [29]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | chatModel | output_parser

In [30]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'A curious fact about Roman Abramovich, the Russian billionaire and former owner of Chelsea Football Club, is that he has a background that includes not only business and sports but also a fascination with art. Abramovich is known for his extensive art collection, which features works by renowned artists such as Francis Bacon and Jean-Michel Basquiat. His interest in art led him to establish the Garage Museum of Contemporary Art in Moscow, a significant cultural institution that promotes contemporary art in Russia. This aspect of his life showcases a different side of Abramovich, beyond his business dealings and football interests.'

* As you saw, the prompt Runnable took "Abramovich", the output of the RunnableParallel, as the value for the "soccer_player" variable.